# **Train Notebook**

In [61]:
import os
import datetime
import sagemaker
from sagemaker.pytorch.estimator import PyTorch

In [62]:
BUCKET = 'enel-noprod-glin-ap35001-odin1'
ROLE = 'arn:aws:iam::856657991664:role/GG-SH-AWS-Enel-GLINSviCol-glin-ap35001_sb_reduce'
TAGS = [
    {'Key': 'sc', 'Value': 'glin'},
    {'Key': 'Gias_ID', 'Value': 'AP35001'},
    {'Key': 'env', 'Value': 'sb'},
]
PREFIX = 'glin-ap35001-sb-smtj-'

In [63]:
S3_DATASET = "s3://enel-noprod-glin-ap35001-sandbox/ODIN/S0010639/transreid/dataset/Market1501/"

In [64]:
DATE = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
MODEL_NAME = f"lighting-reid-market-{DATE}"
TRAIN_BASE_JOB_NAME = f'{PREFIX}{MODEL_NAME}'
TRAIN_JOB_NAME = f'{TRAIN_BASE_JOB_NAME}'
OUTPUT_DIR = f"s3://enel-noprod-glin-ap35001-sandbox/ODIN/S0010639/transreid/output/{TRAIN_JOB_NAME}"

In [65]:
print(f"Model name {MODEL_NAME}")
print(f"Train Job Name {TRAIN_JOB_NAME}")
print(f"Output S3 dir {OUTPUT_DIR}")

Model name lighting-reid-market-2022-07-15-19-17-58
Train Job Name glin-ap35001-sb-smtj-lighting-reid-market-2022-07-15-19-17-58
Output S3 dir s3://enel-noprod-glin-ap35001-sandbox/ODIN/S0010639/transreid/output/glin-ap35001-sb-smtj-lighting-reid-market-2022-07-15-19-17-58


In [66]:
TRAIN_INPUTS = {
    "dataset": S3_DATASET
}
print(TRAIN_INPUTS)

{'dataset': 's3://enel-noprod-glin-ap35001-sandbox/ODIN/S0010639/transreid/dataset/Market1501/'}


In [67]:
train_estimator = PyTorch(
    source_dir ="../",
    entry_point="train.py",
    sagemaker_session=sagemaker.Session(default_bucket=BUCKET), 
    role=ROLE,
    tags=TAGS,
    py_version='py38',
    framework_version='1.10.0',
    instance_type="ml.p3.8xlarge",
    instance_count=1,
    volume_size=100,
    base_job_name=TRAIN_BASE_JOB_NAME,
#    hyperparameters=TRAIN_HYPERPARAMS,
    output_path=f"{OUTPUT_DIR}/output",
    checkpoint_local_path='/opt/ml/checkpoints/',
    checkpoint_s3_uri=f"{OUTPUT_DIR}/checkpoints",
    debugger_hook_config=False,
    code_location=f"{OUTPUT_DIR}/code"
)

In [68]:
train_estimator.fit(
    inputs=TRAIN_INPUTS,
    job_name=TRAIN_JOB_NAME,
    wait=True
)

2022-07-15 19:18:09 Starting - Starting the training job...ProfilerReport-1657912689: InProgress
...
2022-07-15 19:18:56 Starting - Preparing the instances for training......
2022-07-15 19:20:00 Downloading - Downloading input data...........................................................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-15 19:29:57,703 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-15 19:29:57,743 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-15 19:29:57,749 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-15 19:29:58,403 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
Preparing metadata (setup.py): started
Preparing metadata (setup.py): finished with status 'done'
Prep

UnexpectedStatusException: Error for Training job glin-ap35001-sb-smtj-lighting-reid-market-2022-07-15-19-17-58: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!
"
Command "/opt/conda/bin/python3.8 train.py", exit code: 1